## Basic RL usage

### Initializing environments


#### **Environment settings**

Initializing an environment is done with the `BaseEnv` class. The `BaseEnv` class leverages the `nocturne` simulator to create a basic RL interface, based on the provided traffic scenarios. 

---
> 📝 The `env_config.yaml` file defines our environment settings, such as the action space, observation space and traffic scenarios to use.
---

Check out `configs/env_config` for all the details!

In [32]:
import yaml
from nocturne.envs.base_env import BaseEnv

# Load environment settings
with open(f"../configs/env_config.yaml", "r") as stream:
    env_config = yaml.safe_load(stream)

# Initialize environment
env = BaseEnv(
    config=env_config,
)

In [33]:
print(f'controlling agents # {[agent.id for agent in env.controlled_vehicles]}')

controlling agents # [32, 3]


#### **Data**

Within our `env_config.yaml`, we specify the path to the folder containing the traffic scenarios to use as follows:

```yaml
# Path to folder with traffic scene(s) from which to create an environment
data_path: ../data
```

[Here](https://github.com/facebookresearch/nocturne/tree/main#downloading-the-dataset) are the instructions to access the complete dataset of traffic scenes. 

The data folder also has a file named `valid_files.json`. This file lists the names of all the valid traffic scenarios along with the ids of the vehicles that are not valid. These vehicles are excluded from our experiment.

For simplicity, we currently added a single traffic scenario that includes two vehicles in our data folder. Both vehicles can be used, so our `valid_files.json` looks like this:

```yaml
{
    "example_scenario.json": []
}
```

### Interacting with the environment

The classic agent-environment loop of reinforcement learning is implemented as follows:

In [53]:
# Reset
obs_dict = env.reset()
# Get info
agent_ids = [agent_id for agent_id in obs_dict.keys()]
dead_agent_ids = []
num_agents = len(agent_ids)
rewards = {agent_id: 0 for agent_id in agent_ids}

for step in range(1000):

    # Sample actions
    action_dict = {
        agent_id: env.action_space.sample() 
        for agent_id in agent_ids
        if agent_id not in dead_agent_ids
    }
    
    # Step in env
    obs_dict, rew_dict, done_dict, info_dict = env.step(action_dict)

    for agent_id in action_dict.keys():
        rewards[agent_id] += rew_dict[agent_id]

    # Update dead agents
    for agent_id, is_done in done_dict.items():
        if is_done and agent_id not in dead_agent_ids:
            dead_agent_ids.append(agent_id)

    # Reset if all agents are done
    if done_dict["__all__"]:
        print(f'Done after {env.step_num} steps -- total return in episode: {rewards}')
        obs_dict = env.reset()
        dead_agent_ids = []
        rewards = {agent_id: 0 for agent_id in agent_ids}

# Close environment
env.close()

Done after 80 steps -- total return in episode: {32: 2.2131572996676643, 3: 1.99559019886293}
Done after 80 steps -- total return in episode: {32: 2.239125186173855, 3: 1.515264166075405}
Done after 80 steps -- total return in episode: {32: 2.111057987645173, 3: -0.28087208758880106}
Done after 78 steps -- total return in episode: {32: 10.295581054315042, 3: 9.067465782257736}
Done after 80 steps -- total return in episode: {32: 2.3630561843629114, 3: 1.4357060624685754}
Done after 72 steps -- total return in episode: {32: 1.6086314833639868, 3: 10.145121757017481}
Done after 80 steps -- total return in episode: {32: 0.9865223892898515, 3: 2.227045011692687}
Done after 80 steps -- total return in episode: {32: 2.3417453472982963, 3: 0.7922847532241497}
Done after 80 steps -- total return in episode: {32: 1.3733428712017077, 3: 1.489925479607083}
Done after 50 steps -- total return in episode: {32: 1.2751634816741446, 3: 8.812925431540755}
Done after 80 steps -- total return in episode:

### Accessing information about the environment

In [55]:
# The observation space 
env.observation_space


Box(-inf, inf, (10,), float32)

In [56]:
# The size of the joint action space 
env.action_space


Discrete(9)

In [31]:
# Which agents are controlled?
env.controlled_vehicles

[<nocturne_cpp.Vehicle at 0x13579ccf0>, <nocturne_cpp.Vehicle at 0x13579ebb0>]

### 
